In [56]:
import nltk 
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import nltk.corpus

import spacy

import gensim
from gensim import corpora
from gensim.models import CoherenceModel

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

import pandas as pd
import numpy as np

import string
import csv
import datetime

In [2]:
class Metrics:
    @staticmethod
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    @staticmethod
    def weighted_mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs(((y_true*y_pred) - (y_pred*y_pred)) / (y_true* y_pred))) * 100

In [9]:
class Research:
    @staticmethod
    def parse_xls(xls_file=None):
        if not xls_file:
            return False
        df = pd.read_excel(xls_file)
        return list(df['Abstract Note'])

    @staticmethod
    def parse_scholar(query_word='security', size=6, initial_time=None, end_time=None):
        import scholarly
        q = '/scholar?lr=lang_us&q='+query_word+'&hl=en-US&as_vis=1&as_sdt=1,5'
        if(initial_time != None):
            q = q + '&as_ylo=' + initial_time
        if(end_time != None):
            q = q +'&as_yhi=' + end_time
        searchFilter = scholarly.search_pubs_custom_url(q)
        return [next(searchFilter).bib['abstract'] for i in range(size)]
    @staticmethod
    def words_stop(stopPath):
        stop_words = set(stopwords.words('english'))
        with open(stopPath, "rb") as msw:
            my_stops = msw.read().decode('utf-8').split("\r\n")            
            ret = stop_words | set(my_stops)
        return ret
    @staticmethod    
    def clean(doc, stopPath):
        exclude = set(string.punctuation)
        lemma = WordNetLemmatizer()          
        stop = set(stopwords.words('english'))      
        my_stops = Research.words_stop(stopPath)
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return " ".join([i for i in normalized.split() if i not in my_stops])
    @staticmethod
    def save(path, doc_list):
        with open(path, "wb") as file:
            doc_str = str(doc_list).encode("utf-8")
            file.write(doc_str)

In [4]:
class Cleaner:
    def __init__(self):
        self.result = []
    @staticmethod
    def clean_xls(xls_file_in=None, xls_file_out=None):
        if not xls_file_in:
            return False
        if not xls_file_out:
            xls_file_out = xls_file_in

        data = pd.read_excel(xls_file_in, index_col=0)
        data = clean_panda(data)
        data.to_excel(xls_file_out)
        
    @staticmethod
    def clean_csv(csv_file_in=None, csv_file_out=None):
        if not csv_file_in:
            return False
        if not csv_file_out:
            csv_file_out = csv_file_in

        data = pd.read_csv(csv_file_in, index_col=0)
        data = clean_panda(data)
        data.to_csv(csv_file_out)

    @staticmethod
    def clean_panda(data):
        data["abstract"] = data["abstract"].apply(lambda x: re.sub("([©]*)\.","",x))#remove copyright's
        data["abstract"] = data["abstract"].apply(lambda x: re.sub('\S*@\S*\s?', '', x))#remove emails
        data["abstract"] = data["abstract"].apply(lambda x: re.sub('\s+', ' ', x))#
        data["abstract"] = data["abstract"].apply(lambda x: re.sub("\'", ' ', x))# remove '
        return data

__Tokenization__ : Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.<br><br>
Words that have __fewer than 3 characters__ are removed.<br><br>
All __stopwords__  are removed.<br><br>
Words are __lemmatized__  —  words in third person are changed to first person and verbs in past and future tenses are changed into present.<br><br>
Words are __stemmed__  —  words are reduced to their root form.

In [5]:
#Set variables
source = 'Papers.xlsx'
stopWords = 'BlockWords.txt'
clean_path = 'CleanArchive.txt'
log_path = 'BeforeArchive.txt'
topics = 10
words = 8

In [10]:
#prepare docs
#log = research.revoke(source)
doc_complete = Research.parse_xls(source)
doc_clean = [Research.clean(doc, stopWords).split() for doc in doc_complete]


In [11]:
#save this version and alterations
Research.save(log_path, []) #log)
Research.save(clean_path, doc_clean)

In [12]:
#dictionary and matrix
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [13]:
#lda model
LDA = gensim.models.ldamodel.LdaModel

In [130]:
#LDA result
result = LDA(doc_term_matrix, chunksize=604, num_topics = topics, id2word=dictionary, passes = 2, iterations = 100) 

In [131]:
for idx, topic in result.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.014*"de" + 0.013*"isoiec" + 0.011*"standard" + 0.009*"27001" + 0.006*"ism" + 0.006*"audit" + 0.006*"e" + 0.005*"paper" + 0.005*"software" + 0.005*"iso"
Topic: 1 
Words: 0.040*"informationsecurity" + 0.018*"security" + 0.015*"organization" + 0.010*"awareness" + 0.010*"study" + 0.009*"paper" + 0.009*"ism" + 0.009*"factor" + 0.007*"culture" + 0.006*"organizational"
Topic: 2 
Words: 0.015*"security" + 0.010*"compliance" + 0.008*"information" + 0.008*"informationsecurity" + 0.007*"isp" + 0.006*"framework" + 0.006*"approach" + 0.006*"organization" + 0.006*"paper" + 0.005*"federal"
Topic: 3 
Words: 0.016*"ism" + 0.016*"informationsecurity" + 0.014*"business" + 0.011*"paper" + 0.008*"company" + 0.008*"standard" + 0.008*"management" + 0.007*"approach" + 0.006*"user" + 0.006*"need"
Topic: 4 
Words: 0.024*"cloud" + 0.018*"security" + 0.016*"risk" + 0.010*"informationtechnology" + 0.010*"management" + 0.009*"service" + 0.009*"analysis" + 0.008*"bcm" + 0.007*"computing" + 0.007*"

## Metrics works

In [19]:
%%latex
\begin{align}
    Mean\ Absolute\ Error\\
    & MAE &= &\frac{1}{n}\sum_{t=1}^{n}|e_t|\\
    Mean\ Absolute\ Percentage\ Error\\
    & MAPE &= &\frac{100\%}{n}\sum_{t=1}^{n}\left |\frac{e_t}{y_t}\right|\\
    Weighted\ Mean\ Absolute\ Percentage\ Error\\
    & MAPE &= &\frac{100\%}{n}\sum_{t=1}^{n}\left |\frac{e_t - y_t}{y_t}\right|\\
\end{align}

<IPython.core.display.Latex object>

In [ ]:
#Na prática (Corpus =/= Corpus de formação inicial), mas usamos o mesmo aqui para simplificar.
#O modelo pode ser atualizado (treinado) com novos documentos.
other_corpus = common_corpus

lda.update(other_corpus)

In [22]:
corpus = [["a", "a", "b"], ["a", "c"]]
dct = Dictionary(corpus)
dct.doc2idx(["a", "a", "c", "not_in_dictionary", "c"])
# [0, 0, 2, -1, 2]

[0, 0, 2, -1, 2]

### A persistência do modelo é obtida através dos métodos load () e save ().

## Parâmetros:
##### corpus ({iterável da lista de (int, float), scipy.sparse.csc}, opcional) 
- Fluxo de vetores de documento ou matriz esparsa de forma (num_terms, num_documents). Se não for fornecido, o modelo será deixado sem treinamento (presumivelmente porque você deseja chamar update () manualmente).

##### num_topics (int, optional) 
- O número de tópicos latentes solicitados a serem extraídos do corpus de treinamento.

#####  id2word ({dict of (int, str), gensim.corpora.dictionary.Dictionary}) 
- Mapeamento de IDs de palavras em palavras. Ele é usado para determinar o tamanho do vocabulário, bem como para depuração e impressão de tópicos.

#####  distributed (bool, optional) 
- Se a computação distribuída deve ser usada para acelerar o treinamento.

##### chunksize (int, optional) 
- Número de documentos a serem usados ​​em cada bloco de treinamento.

#####  passes (int, opcional) 
- Número de passagens pelo corpus durante o treinamento.

#####  update_every (int, opcional) 
- Número de documentos a serem repetidos para cada atualização. Definido como 0 para aprendizado em lote,> 1 para aprendizado interativo iterativo.

#####  alpha ({numpy.ndarray, str}, opcional) 
- Pode ser definido para um array 1D de comprimento igual ao número de tópicos esperados que expressam nossa crença a priori para a probabilidade de cada tópico. Alternativamente, as estratégias de seleção prévia padrão podem ser empregadas fornecendo uma string:

>  'Assimétrico': Usa um prior assimétrico fixo normalizado de 1.0 / topicno.

>  'Auto': aprende um prior assimétrico do corpus (não disponível se distribuído == True).

#####  eta ({float, np.array, str}, opcional) - A priori crença na probabilidade da palavra, isso pode ser:
- escalar para um prior simétrico sobre probabilidade de tópico / palavra, vetor de comprimento num_words para denotar uma probabilidade definida pelo usuário assimétrica para cada palavra, matriz de forma (num_topics, num_words) para atribuir uma probabilidade para cada combinação de tópicos de palavras,  a string "auto" para aprender a priori assimétrica a partir dos dados.

##### decay (float, optional) 
- Um número entre (0.5, 1) para pesar que porcentagem do valor lambda anterior é esquecida quando cada novo documento é examinado Corresponde ao Kappa de Matthew D. Hoffman, David M. Blei e Francis Bach: “Aprendizagem on-line para a alocação de Dirichlet Latent NIPS'10”.

##### offset (flutuante, opcional)
- Hyper-parâmetro que controla o quanto vamos desacelerar os primeiros passos nas primeiras iterações. Corresponde a Tau_0 de Matthew D. Hoffman, David M. Blei e Francis Bach: “Aprendizagem on-line para alocação de Dirichlet latente NIPS '10”.

##### eval_every (int, opcional) 
- A perplexidade do log é estimada em cada uma das muitas atualizações. Definir isso para um diminui o treinamento em ~ 2x.

##### iterações (int, opcional) 
- Número máximo de iterações no corpus ao inferir a distribuição de tópico de um corpus.

##### gamma_threshold (float, optional) 
- Mínima alteração no valor dos parâmetros gama para continuar a iteração.

##### minimum_probability (float, opcional) 
- Tópicos com uma probabilidade menor que esse limite serão filtrados.

##### random_state ({np.random.RandomState, int}, opcional)
- Um objeto randomState ou um seed para gerar um. Útil para reprodutibilidade.

##### ns_conf (dict de (str, objeto), opcional)
- Parâmetros de palavra chave propagados para gensim.utils.getNS () para obter um Pyro4 Nameserved. Usado somente se distribuído estiver definido como True.

##### minimum_phi_value (float, opcional) 
- se per_word_topics for True, isso representa um limite inferior no termo probabilidades.

##### per_word_topics (bool)
- Se for True, o modelo também calcula uma lista de tópicos, classificados em ordem decrescente dos tópicos mais prováveis para cada palavra, juntamente com seus valores phi multiplicados pelo comprimento do recurso (ou seja, contagem de palavras).

##### callbacks (lista de retorno de chamada) 
- Retornos métricos para registrar e visualizar as métricas de avaliação do modelo durante o treinamento.

##### dtype ({numpy.float16, numpy.float32, numpy.float64}, opcional) 
- Tipo de dados a ser usado durante os cálculos dentro do modelo. Todas as entradas também são convertidas.

gensim.models.ldamodel.LdaModel
(corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, minimum_probability=0.01, random_state=None, ns_conf=None, minimum_phi_value=0.01, per_word_topics=False, callbacks=None, dtype=<type 'numpy.float32'>)


In [47]:
result.state.numdocs

604

In [139]:
table = result.show_topics(formatted=False)

In [140]:
predict = []
for x in range(10):
    for y in range(10):
        predict.append(table[x][1][y][1])
predict[:6]

[0.01436848, 0.012892991, 0.011017632, 0.009469971, 0.006496887, 0.0058038095]

In [111]:
id2word = corpora.Dictionary(doc_clean)
corpus = [id2word.doc2bow(text) for text in doc_clean]
print('\nPerplexity: ', result.log_perplexity(corpus))


Perplexity:  -8.700897277369512


In [123]:
result.top_topics(corpus)

[([(0.048588727, 'informationsecurity'),
   (0.024742568, 'organization'),
   (0.021666674, 'study'),
   (0.020922206, 'ism'),
   (0.019962503, 'model'),
   (0.017341344, 'research'),
   (0.014328179, 'analysis'),
   (0.013171633, 'approach'),
   (0.012307799, 'paper'),
   (0.011820134, 'process'),
   (0.011669175, 'result'),
   (0.010899328, 'security'),
   (0.010188433, 'framework'),
   (0.010132207, 'awareness'),
   (0.009090984, 'threat'),
   (0.008917793, 'method'),
   (0.008843088, 'issue'),
   (0.008762392, 'application'),
   (0.008473777, 'knowledge'),
   (0.008460383, 'business')],
  -1.464047211783102),
 ([(0.06632323, 'security'),
   (0.030999035, 'information'),
   (0.02586801, 'system'),
   (0.025247037, 'ism'),
   (0.017256139, 'standard'),
   (0.015436605, 'network'),
   (0.012547541, 'control'),
   (0.01147822, 'data'),
   (0.01022834, 'paper'),
   (0.009856113, 'iso'),
   (0.009718649, 'need'),
   (0.008569867, 'level'),
   (0.008132857, 'policy'),
   (0.007661441, 'te

In [135]:
# Compute Coherence Score !lower is better
coherence_model_lda = CoherenceModel(model=result, texts=doc_clean, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3591983721670916


In [132]:
real = [result.alpha for x in range(10)]
real = list(i for j in real for i in j)

In [141]:
print(Metrics.mean_absolute_percentage_error(real,predict))
print(Metrics.weighted_mean_absolute_percentage_error(real,predict))

89.4754409790039
89.4754409790039


In [142]:
print(Metrics.mean_absolute_percentage_error(real,predict))

89.4754409790039
